In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

import logging

logging.basicConfig(level=logging.INFO)

In [ ]:
import numpy as np
import pandas as pd
import torch

from tsdm.encoders.numerical import BoundaryEncoder

In [ ]:
encoder = BoundaryEncoder(0, 1, mode="clip")
data = torch.tensor(np.random.randn(100))
encoder.fit(data)
encoder.encode(data)

In [ ]:
np.where(data <= 1, np.where(data >= 0, data, float("nan")), float("nan"))

In [ ]:
x <= 1